## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_app_df = application_df.drop(columns=["EIN","NAME"])
clean_app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
clean_app_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
appTypeCounts = pd.value_counts(clean_app_df.APPLICATION_TYPE)
appTypeCounts_df = pd.DataFrame(appTypeCounts)
print(appTypeCounts)
appTypeCounts_df.head()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


,APPLICATION_TYPE
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065


In [5]:
appTypeCounts_df.index[1]

'T4'

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`


application_types_to_replace = []
for i in range(len(clean_app_df)):
    for j in range(len(appTypeCounts)):
        if clean_app_df["APPLICATION_TYPE"][i]==appTypeCounts_df.index[j]:
            if appTypeCounts_df["APPLICATION_TYPE"][j]>500:
                # print(f"Greater than 500 - {appTypeCounts_df['APPLICATION_TYPE'][j]}")
                application_types_to_replace.append(appTypeCounts_df.index[j])
            else:
                # print(f"Not greater than 500 - {appTypeCounts_df['APPLICATION_TYPE'][j]}")
                application_types_to_replace.append("Other")

# Replace in dataframe
clean_app_df['APPLICATION_TYPE'] = application_types_to_replace

# # Check to make sure binning was successful
clean_app_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classTypeCounts = pd.value_counts(clean_app_df.CLASSIFICATION)
classTypeCounts_df = pd.DataFrame(classTypeCounts)
print(classTypeCounts)
classTypeCounts_df.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for i in range(len(clean_app_df)):
    for j in range(len(classTypeCounts)):
        if clean_app_df["CLASSIFICATION"][i]==classTypeCounts_df.index[j]:
            if classTypeCounts_df["CLASSIFICATION"][j]>1800:
                # print(f"Greater than 500 - {appTypeCounts_df['APPLICATION_TYPE'][j]}")
                classifications_to_replace.append(classTypeCounts_df.index[j])
            else:
                # print(f"Not greater than 500 - {appTypeCounts_df['APPLICATION_TYPE'][j]}")
                classifications_to_replace.append("Other")

# Replace in dataframe
clean_app_df['CLASSIFICATION'] = classifications_to_replace

# # Check to make sure binning was successful
clean_app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
clean_app_df_encoded = pd.get_dummies(data=clean_app_df, columns=['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS'])
clean_app_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = clean_app_df_encoded.IS_SUCCESSFUL.values
X = clean_app_df_encoded.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu", input_dim=43))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# # Third hidden layer
# nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 20)                880       
                                                                 
 dense_7 (Dense)             (None, 20)                420       
                                                                 
 dense_8 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,321
Trainable params: 1,321
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.5820 - accuracy: 0.7153
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5584 - accuracy: 0.7279
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5544 - accuracy: 0.7291
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7311
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7313
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7321
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7322
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7318
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7325
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5474 - accuracy: 0.7323

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5546 - accuracy: 0.7293 - 482ms/epoch - 2ms/step
Loss: 0.5546241402626038, Accuracy: 0.7293294668197632


In [24]:
# Export our model to HDF5 file

nn_model.save("AlphabetSoupCharity.h5")